In [24]:
from typing import Any, Dict, List, Optional,Set, Tuple 
import numpy as np
import pandas as pd
from random import sample,random
from math import floor
import matplotlib.pyplot as plt
!gdown --id 1xB6ewaR3HMKzhMJgIgvMWHNODifHDHUJ
!gdown --id 1OP0-auufRe8cS_S7KVtiG15XIUglILze
def generate_data(size : int = 100,coefs :int = 4) -> np.array:
    """
    Untuk Dummy saja
    Generate Random data < Continous >
    size = How many observations
    coefs = How many features / how many chromosome
    Returns:
        np array (x), (y)
    """
    coeffs : List[float] = [0.4, 0.3 ,0.2, -0.1]
    x : List[np.array] = [[np.random.normal() for _ in range(len(coeffs))] for _ in range(size) ]
    y : List[np.array] = [np.dot(i,coeffs) for i in x]
    
    return np.array(x), np.array(y)

class Population:
    
    def __init__(self,TotalSize,Totalgenome: int) -> None:
        self.Totalgenome: int = Totalgenome
        self.totalSize : int = TotalSize
        self.bestIndividuals : List =[]
        self.population = []
        self.fitnessed  : Dict[str,Any]

    def createIndividu(self) -> np.array:
        """
        Create Individu using Real Numbers

        Returns:
            List[int]: Real numbers sequence
        """
        return np.random.normal(size=self.Totalgenome)
    
    def createPopulation(self) -> None :
        """
        Create Population with x size Depends on total size
        and append it to population list 
        
        Returns:
            None 
        """
        self.population.append([self.createIndividu() for _ in range(self.totalSize)])
   
    def fitness(self, individual , inputs : np.array, yTrue : np.array) -> Dict[str,Any]:
        """
        
        Fitness current Individual

        Args:
            individual (List[int]) --> Chromosome on a population
            inputs (np.array) ->> Input data
            yTrue (np.array) --> Groud truth
        Returns:
            Dict[str,Any]: [description]
        """
        info : Dict[str,Any] = {}
        bias : np.float = 0
        predicted : np.array =  bias + np.dot(inputs,individual) 
        yTrue_mean : np.float = np.mean(yTrue) 

        SST : np.float = np.sum(np.array([(y - yTrue_mean) ** 2 for y in yTrue]),axis=None)
        SSR : np.float = np.sum(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in zip(yTrue,predicted)]),
                                axis = None)
    
        bias :np.float = np.mean(np.sqrt(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in 
                                                   zip(yTrue,predicted)])))
        RMSE : np.float = np.sqrt(SSR/ len(y))
        Rsquared : np.float = (1 - (SSR / SST))

        SSE : np.float = SSR / len(y)
        fitness : np.float = 1 / len(yTrue) * SSR

        info["Rsquared"] = Rsquared
        info["coeff"] = individual
        info["error"] = SSE
        info["RMSE"] = RMSE 
        info["fitness"] = fitness 
        info["bias"] = bias 
        self.fitnessed = info
        
        return info
    
    def evaluate_population(self, x:np.array ,y:np.array ,selectionSize : int) -> None :
        """
        Function to evaluate the best individual from current population

        Args:
           x (np.array) , Input data
           y (np.array, Ground truth
           selectionSize (int) : How many best Individuals you want to grab from the population

        Returns:
            None
        """
        fitness_list= [self.fitness(individual,x,y) for individual in self.population[0]]
        error_list : List[Dict[str,Any]] = sorted(fitness_list,key=lambda i : i["fitness"])
        best_individuals = error_list[: selectionSize]
        self.bestIndividuals.clear()
        self.bestIndividuals.append(best_individuals)
        


    def mutate(self,individual : List[float], probabilityMutating : float) -> List[float]:
        """
        
        Ini Fungsinya lakuin mutasi 

        Args:
            individual (List[float]): Individu pada populasi
            probabilityMutating (float): Probabilitas Individu saat ini bermutasi 

        Returns:
            List[float]: Individu Yang telah bermutasi < Jika dilakukan >
        """
        indx : List[int] = [i for i in range(len(individual))]

        totalMutatedGens : int = int(probabilityMutating * len(individual))
        indx_toMutate : List[int] = sample(indx,k = totalMutatedGens)
        for ind in indx_toMutate:
            choice : np.int = np.random.choice([-1,1])
            gene_transform : float = choice*random()
            individual[ind] = individual[ind] + gene_transform
        return individual
        
    def crossover(self, parent1 : Dict[str,Any], parent2 : Dict[str,Any]) -> List[Any]:
        """
           CrossOver / persilangan untuk menghasilkan keturunan 
           metode yang digunakan adalah Simple Arithmetic crossover
        Args:
            parent1 (Dict[str,Any]): [description]
            parent2 (Dict[str,Any]): [description]
        Return: None
        """
        
        anak_keturunan : Dict[int,Any] = {}
        index : List[int] = [i for i in range( self.Totalgenome )]
        indexRandomize : List[int] = sample(index, floor(0.5 * self.Totalgenome))
        IndexNotInRandomize : List[int] = [i for i in index if i not in indexRandomize]

        getCromosomeFromParent1 : List[Any] = [[i,parent1['coeff'][i]] for i in indexRandomize]
        getCromosomeFromParent2 : List[Any] = [[i,parent2["coeff"][i]] for i in IndexNotInRandomize]
        
        anak_keturunan.update({key :value for (key,value) in getCromosomeFromParent1})
        anak_keturunan.update({key : value for(key,value) in getCromosomeFromParent2})
    
        return [anak_keturunan[i] for i in index]

    def create_new_generation(self,probabilityCrossOver: float,probability_indiv_mutating:float) -> None:
        """
        Ini method untuk pergantian generasi Menggunakan generational model
        Create new population using the best individuals < elitisime >, self.bestIndividuals adalah
        Best individual

        Args:
            probabilityMutating(float) : 
            probability_indiv_mutating (float) : 

        Return:
            None
        """
        pasangan_sah = [sample(self.bestIndividuals[0],2) for _ in range( self.totalSize)]
        crossOverered_parents = [self.crossover(pasangan[0],pasangan[1]) for pasangan in pasangan_sah]
        pasangan_sah_indx = [i for i in range(self.totalSize)]
        pasanganCalonMutasi = sample(pasangan_sah_indx,floor(probabilityCrossOver* self.totalSize))
        PasanganMutasi = [[i,self.mutate(crossOverered_parents[i],probability_indiv_mutating)] 
                          for i in pasanganCalonMutasi]
        for anakMutasi in PasanganMutasi:
            crossOverered_parents[anakMutasi[0]] = anakMutasi[1]
        self.population.clear()
        self.population.append(crossOverered_parents)
    
if __name__ == "__main__":
    x,y = generate_data(100)
    np.random.seed(123) 
    pop =Population(100,4)
    pop.createPopulation()
    selectionSize = floor(0.1 * 100)
    max_generations = 50

    probability_crosss_over = 0.8
    probability_indiv_mutation = 0.25

    for i in range(3) :
        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])
    

    #parent1 = pop.fitness(individual,x,y)
    #parent2 = pop.fitness(individual,x,y)

'gdown' is not recognized as an internal or external command,
operable program or batch file.


Nilai x :  [ 0.3115815  -0.14205073  0.75203367  0.06964075]
Nilai Y :  0.2254600419193178
Predicted =  0.28808355926332113
{'Rsquared': 0.9664374444784752, 'coeff': [0.3912198122602317, 0.2796626701579614, 0.1771923291989879, -0.19911238275725224], 'error': 0.011263096474426314, 'RMSE': 0.1061277365933445, 'fitness': 0.011263096474426316, 'bias': 0.08652472590362892}


'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
def terminate(ans : Dict[str,Any]):
    """
    Terminate Genetic Algorithm Menggunakan nilai RSquared yang telah didefinisiskan
    
    ans : Dict - > Berisi nilai Rsquared,coeff,RMSE ...
    
    return 
    True jika Rsquared > 0.95
    False others
    """
    if ans["Rsquared"] > 0.98:
        return True
    else :
        return False

In [3]:
from sklearn.preprocessing import StandardScaler
StdScaler = StandardScaler()


In [17]:
"""
Codingan Ini digunakan untuk preprocessing data saham dari link berikut : https://github.com/wildangunawan/Dataset-Saham-IDX
menjadi nilai open selama 10 hari berturut turut
"""
df = pd.read_csv("saham.csv")
dfnew = pd.DataFrame()
yow = []
def generate_newdataset():
    k=13
    j=1
    for i in range(100):
        
        dict1 = {}
        dict1.update( {i:df[j:k]["open_price"].values})
        j+=1
        k+=1
        yow.append(dict1)
generate_newdataset()
dfnew = pd.DataFrame(yow[0])
for i in range(1,len(yow)):
    dfnew[i] = pd.DataFrame(yow[i])
dfnew = dfnew.T
dfnew.to_csv("DatasetRequired.csv")


In [25]:
if __name__ == "__main__":
    df = pd.read_csv("DatasetRequired.csv")

    df = df.iloc[:,1:13]
    x = df.iloc[:,1:12]
    y = df.iloc[:,11]
    
    realY =y.copy()
    y = StdScaler.fit_transform(np.array(y).reshape(-1,1))
    
    x = StdScaler.fit_transform(x) 
    
    np.random.seed(123) 
    pop =Population(100,11)
    pop.createPopulation()
    selectionSize = floor(0.15 * 100)

    probability_crosss_over = 0.95
    probability_indiv_mutation = 0.1
    
    terminate_ga = False
    generation = 0
    while(terminate_ga != True): 
        if(generation == max_generations):
            break
        print(f"generation : {generation}")

        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        terminate_ga = terminate(pop.bestIndividuals[0][0])
        print(f"RSquared : {pop.bestIndividuals[0][0]['Rsquared']}")
        generation += 1
        
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])

generation : 0
RSquared : -0.06969583240969945
generation : 1
RSquared : 0.710279665564379
generation : 2
RSquared : 0.844418129846905
generation : 3
RSquared : 0.8119759977733292
generation : 4
RSquared : 0.9164671168679952
generation : 5
RSquared : 0.9034413705091797
generation : 6
RSquared : 0.9350646612084932
generation : 7
RSquared : 0.923286840537723
generation : 8
RSquared : 0.9529131182348675
generation : 9
RSquared : 0.9573384828309082
generation : 10
RSquared : 0.9745970244808789
generation : 11
RSquared : 0.9719727877501753
generation : 12
RSquared : 0.9789198511589694
generation : 13
RSquared : 0.9840518312754669
Nilai x :  [2.75086792 2.86571695 2.32914034 1.99572509 0.87738216 0.74882796
 1.29885108 1.43537095 1.41540623 1.54884825 1.37563292]
Nilai Y :  [1.37563292]
Predicted =  1.3901199924836662
{'Rsquared': 0.9840518312754669, 'coeff': [-0.23600807938766918, 0.1532090998681237, -0.037486064296003985, 0.14683222783822802, -0.04304177578064844, 0.06867678194976001, -0.0

In [26]:
StdScaler.fit_transform(np.array(realY).reshape(1,-1))
predictedReal = StdScaler.inverse_transform(pop.bestIndividuals[0][0]['bias'] + np.dot(x, pop.bestIndividuals[0][0]["coeff"]))
predicted = pop.bestIndividuals[0][0]['bias'] + np.dot(x, pop.bestIndividuals[0][0]["coeff"])
for i in range(len(y)):
    print("Hari ke - %d, nilai truth open %.4f , nilai prediksi open %.4f "% (i+1, realY[i],predictedReal[i]))

Hari ke - 1, nilai truth open 7500.0000 , nilai prediksi open 7501.3901 
Hari ke - 2, nilai truth open 7350.0000 , nilai prediksi open 7350.7058 
Hari ke - 3, nilai truth open 7400.0000 , nilai prediksi open 7400.8379 
Hari ke - 4, nilai truth open 7450.0000 , nilai prediksi open 7451.0996 
Hari ke - 5, nilai truth open 7425.0000 , nilai prediksi open 7426.2711 
Hari ke - 6, nilai truth open 7325.0000 , nilai prediksi open 7326.0892 
Hari ke - 7, nilai truth open 7300.0000 , nilai prediksi open 7300.8281 
Hari ke - 8, nilai truth open 7250.0000 , nilai prediksi open 7250.6813 
Hari ke - 9, nilai truth open 7100.0000 , nilai prediksi open 7100.1978 
Hari ke - 10, nilai truth open 7050.0000 , nilai prediksi open 7049.9448 
Hari ke - 11, nilai truth open 7025.0000 , nilai prediksi open 7024.8853 
Hari ke - 12, nilai truth open 7050.0000 , nilai prediksi open 7050.0778 
Hari ke - 13, nilai truth open 7125.0000 , nilai prediksi open 7125.2730 
Hari ke - 14, nilai truth open 7225.0000 , nila